Data Cleaning and preparation completed

In [10]:
{
  "id": "global_skills_development_report",
  "name": "Global Skills Development Report",
  "url": "https://www.shl.com/products/product-catalog/view/global-skills-development-report/",
  "test_type": [
    "Aptitude",
    "Behavioral",
    "Cognitive",
    "Development",
    "English",
    "Personality & Behavior"
  ],
  "text_for_embedding": "Assessment name: Global Skills Development Report. Assessment types: Aptitude, Behavioral, Cognitive, Development, English, Personality & Behavior. Used for hiring and evaluating candidates based on skills, abilities, and behavior."
}


{'id': 'global_skills_development_report',
 'name': 'Global Skills Development Report',
 'url': 'https://www.shl.com/products/product-catalog/view/global-skills-development-report/',
 'test_type': ['Aptitude',
  'Behavioral',
  'Cognitive',
  'Development',
  'English',
  'Personality & Behavior'],
 'text_for_embedding': 'Assessment name: Global Skills Development Report. Assessment types: Aptitude, Behavioral, Cognitive, Development, English, Personality & Behavior. Used for hiring and evaluating candidates based on skills, abilities, and behavior.'}

In [11]:
import pandas as pd

In [12]:
data = pd.read_json('Raw_data.json')
df= pd.DataFrame(data)
df.head()
df["test_type"]
df

,name,url,remote_testing,adaptive_irt,test_type
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,False,False,"[A, E, B, C, D, P]"
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,False,False,[K]
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
...,...,...,...,...,...
365,Written English v1,https://www.shl.com/products/product-catalog/v...,False,False,[K]
366,Written Spanish,https://www.shl.com/products/product-catalog/v...,False,False,[K]
367,Zabbix (New),https://www.shl.com/products/product-catalog/v...,False,False,[K]
368,360 Digital Report,https://www.shl.com/products/product-catalog/v...,False,False,[D]


In [13]:
TEST_TYPE_MAP = {
    "A": "Aptitude",
    "B": "Behavioral",
    "C": "Cognitive",
    "D": "Development",
    "E": "English",
    "F": "Functional",
    "K": "Knowledge & Skills",
    "P": "Personality & Behavior"
}



In [14]:
TEST_TYPE_MAP = {
    "A": "Aptitude",
    "B": "Behavioral",
    "C": "Cognitive",
    "D": "Development",
    "E": "English",
    "F": "Functional",
    "K": "Knowledge & Skills",
    "P": "Personality & Behavior"
}

df["test_type"] = df["test_type"].apply(
    lambda codes: [TEST_TYPE_MAP[c] for c in codes if c in TEST_TYPE_MAP]
)


In [15]:
def build_text_for_embedding(row):
    # --- Test types ---
    if isinstance(row["test_type"], list):
        test_types = ", ".join(row["test_type"])
    else:
        test_types = str(row["test_type"])

    # --- Flags ---
    remote = "supports remote testing" if row.get("remote_testing", False) else "on-site testing"
    adaptive = (
        "uses adaptive IRT methodology"
        if row.get("adaptive_irt", False)
        else "uses standard testing methodology"
    )

    # --- Skill & intent expansion ---
    skills_hint = test_types.replace("&", "and")

    return (
        f"Assessment name: {row['name']}. "
        f"This is an SHL assessment designed for recruitment and hiring. "
        f"It is used to screen job applicants and evaluate candidates. "
        f"Measures skills related to {skills_hint}. "
        f"Assessment category includes {test_types}. "
        f"It {remote} and {adaptive}. "
        f"Suitable for recruiters, hiring managers, and talent acquisition teams "
        f"to assess candidate suitability for job roles."
    )


In [16]:
df["text_for_embedding"] = df.apply(build_text_for_embedding, axis=1)


In [17]:
df["text_for_embedding"].head

<bound method NDFrame.head of 0      Assessment name: Global Skills Development Rep...
1      Assessment name: .NET Framework 4.5. This is a...
2      Assessment name: .NET MVC (New). This is an SH...
3      Assessment name: .NET MVVM (New). This is an S...
4      Assessment name: .NET WCF (New). This is an SH...
                             ...                        
365    Assessment name: Written English v1. This is a...
366    Assessment name: Written Spanish. This is an S...
367    Assessment name: Zabbix (New). This is an SHL ...
368    Assessment name: 360 Digital Report. This is a...
369    Assessment name: 360° Multi-Rater Feedback Sys...
Name: text_for_embedding, Length: 370, dtype: object>

In [18]:
df["test_type"].apply(len).value_counts()


test_type
1    327
0     25
2     14
3      3
6      1
Name: count, dtype: int64

In [19]:
import re

def generate_id(name):
    name = name.lower()
    name = re.sub(r"[^a-z0-9\s]", "", name)
    return re.sub(r"\s+", "_", name)

df["id"] = df["name"].apply(generate_id)


In [20]:
df["id"] = df["name"].apply(generate_id)

df = df[["id"] + list(df.columns.drop("id"))]
len(df)


370

In [21]:
df.to_json("final_assessments.json", orient="records", indent=2)
